In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
%config IPCpmpleter.greedy=True

In [ ]:
train = pd.read_csv('Data Mining VU data/training_set_VU_DM_2014.csv')
test = pd.read_csv('Data Mining VU data/test_set_VU_DM_2014.csv')

In [ ]:
train_sample = train.sample(frac=0.5, random_state=0)
test_sample = train.drop(train_sample.index)

In [ ]:
train_sample.booking_bool.sum()/train_sample.shape[0]

In [ ]:
train.booking_bool.sum()/train.shape[0]

In [ ]:
train.describe()

In [ ]:
# number of different users
len(test.srch_id.unique())

In [ ]:
# numbers of different hotels
len(test.prop_id.unique())

In [ ]:
train.isna().sum()

In [ ]:
train.isna().sum()/train.shape[0]

In [ ]:
numerical_data_of_interest = ['visitor_hist_starrating','visitor_hist_adr_usd','prop_starrating','prop_review_score','prop_brand_bool','prop_location_score1','prop_location_score2','prop_log_historical_price','position','price_usd','promotion_flag','srch_length_of_stay','srch_booking_window','srch_adults_count','srch_children_count','srch_room_count','srch_saturday_night_bool','srch_query_affinity_score','orig_destination_distance','random_bool','click_bool','gross_bookings_usd','booking_bool']
# Compute the correlation matrix
#corr = train.drop(['srch_id','date_time','site_id','visitor_location_country_id','prop_country_id','prop_id','srch_destination_id'], axis=1).corr()
corr = train[numerical_data_of_interest].corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 12))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

In [ ]:
# check if test set contains other users than the training set
test_ids = set(test.srch_id.unique())
train_ids = set(train.srch_id.unique())
intersection_count = len(test_ids & train_ids)
intersection_count == len(test_ids)
print(intersection_count, ' of ',len(test_ids),' users in the test set overlap with the training set')

In [ ]:
# check if test set contains other hotels than the training set
test_ids = set(test.prop_id.unique())
train_ids = set(train.prop_id.unique())
intersection_count = len(test_ids & train_ids)
intersection_count == len(test_ids)
print(intersection_count, ' of ',len(test_ids),' hotels in the test set overlap with the training set')

In [ ]:
train[['booking_bool','prop_id']].groupby(['prop_id'], as_index=False).mean().sort_values(by='prop_id',ascending=False)

In [ ]:
train[train['srch_destination_id']==3][['click_bool','prop_id']].groupby(['prop_id'], as_index=False).mean().sort_values(by='click_bool',ascending=False)

In [ ]:
train_sample['star_diff'] = np.abs(train_sample.visitor_hist_starrating - train_sample.prop_starrating)

In [ ]:
sns.distplot(train_sample.star_diff[train_sample.booking_bool == 1].dropna(), kde=False, bins=15)
#sns.distplot(train_sample.star_diff[train_sample.booking_bool == 1].dropna(), kde=False, bins=15)
plt.show()

In [ ]:
train_sample['price_diff'] = np.abs(train_sample.visitor_hist_adr_usd - train_sample.price_usd)

In [ ]:
sns.distplot(train_sample.price_diff[train_sample.booking_bool == 1].dropna(), kde=False, bins=15)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(train_sample[['']])
kmeans.labels_

In [ ]:
train.visitor_location_country_id.value_counts()

In [ ]:
groups = train.groupby(['prop_country_id','site_id'])

In [ ]:
groups.size().min()

In [ ]:
groups.size()

In [ ]:
groups.apply(lambda x: x.visitor_hist_starrating.mean())

In [ ]:
# make families of children and adults

train['family_size'] = train['srch_children_count'] + train['srch_adults_count']
train.describe()

In [ ]:
train_sample['family_bins'] = train_sample.family_size.apply(lambda x: 'alone' if x==1 else ('couple' if x==2 else ('small family' if x < 5 else 'large family')))
#train['family_bins'].groupby(['family_bins'])
#train_sample.plot(x='family_bins', y='srch_length_of_stay', style='o')
#sns.boxplot(x='booking_bool', y='family_bins', data=train_sample)
sns.distplot(train_sample.family_bins[train_sample.booking_bool==0].dropna(), label='not booked')
sns.distplot(train_sample.family_bisn[train_sample.booking_bool==1].dropna(), label='booked')
plt.legend()
plt.show()

In [ ]:
sns.lmplot('prop_starrating', 'prop_log_historical_price', data=train_sample, fit_reg=False)
#sns.kdeplot(train.prop_starrating, train.prop_log_historical_price)
#train.plot(x='prop_log_historical_price', y='prop_starrating', kind='bar')
plt.show()

In [ ]:
#train.display('prop_country_id')